<a href="https://colab.research.google.com/github/cyruskavwele12/cyrus-Github/blob/master/All_Transect_GLM_Tweedie_distribution_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
test_import <-system('python3 -c "import google.colab"', intern=TRUE)
import_status <- attr(test_import,'status')
datadir<-'../data/'
if (is.null(import_status)) { 
    datadir<-'camera-trap-analysis/data/'
    if (!dir.exists('camera-trap-analysis')){
        install.packages("getPass")

        # no error received so we are running in colab
        username <- readline(prompt="enter GitHub username: ")
        password <- getPass("enter password: ")
        Sys.setenv(git_user =  paste(username, password, sep=":"))
        system('git clone https://$git_user@github.com/ctorney/camera-trap-analysis.git')
        
    }
}

SyntaxError: ignored

In [26]:
#Install packages and libriaries 
# install.packages("CARS")
install.packages("GGally")
install.packages("tweedie")
install.packages("statmod")
install.packages("lme4")
install.packages("ggplot2")
install.packages("dplyr")
install.packages("tidyverse")
install.packages("lubridate")
library(readr)  # for read_csv
library(knitr)
library(httr)
library(dplyr)
library(tidyverse)
library(lubridate)
# library(CARS)
library(GGally) # same as pairs but better easthetics
library(ggplot2)
library(tweedie)
library(statmod)
install.packages("gridExtra")
install.packages("effects")
install.packages("lattice")
library(gridExtra)
library(effects)
library(lattice)


NameError: ignored

In [4]:
# Load the dataframe from local disc or GitHub website (already uploaded)
setwd("D:/Projects/PhD_Work/PhD_Docs_Glasgow/PhD_Data/Camera_Trap_Manual_Data/Cleaned_Manupilated_Data/All_transects")
kuka_density_oneday<-read.csv("kuka_density_oneday.csv",header=TRUE)
mbalageti_density_oneday<-read.csv("mbalageti_density_oneday.csv",header=TRUE)

NameError: ignored

In [21]:
# check dimesnions of the two dataframes 
dim(mbalageti_density_oneday);dim(kuka_density_oneday)

NameError: ignored

In [ ]:
head(mbalageti_density_oneday,1)
head(kuka_density_oneday,1)

In [ ]:
# Take note of the different names for the camera number column
# change the camera no name for mabalageti dataset 
colnames(mbalageti_density_oneday)[colnames(mbalageti_density_oneday) == "camera_no"] <- "camera.no"

In [ ]:
# combine the two dataframes

In [ ]:
#rename the object as transects_all_density (Mbalageti and Kuka trnasect dataframes combined)
transects_all_density<-rbind(mbalageti_density_oneday,kuka_density_oneday) 

In [ ]:
# check df dim
dim(transects_all_density)

In [ ]:
# dummify the regime factor to indicator variables 
is.factor(transects_all_density$regime) #check whether is a factor
contrasts(factor(transects_all_density$regime)) # show how the factor will be converted to indicator varible 

In [ ]:
# Examine any possible interaction between variables
interaction.plot(x.factor = transects_all_density$dist_bound,    # variable to plot on x-axis
                 trace.factor = transects_all_density$regime,   # variable to specify "traces"; here, regime
                 col = c("blue4", "red4"),
                 trace.label = ("Regime"),
                 main="One day interval",
                 xlab=("Distance to the Boundary (KM)"),
                 ylab= ("Wildlife Density"),
                 type="l",
                 lwd=3,
                 response = transects_all_density$wildlife_density)

In [ ]:
# levene test for homogeneity with lawstat package
install.packages("lawstat")
library(lawstat)

levene.test(transects_all_density$wildlife_density, transects_all_density$dist_bound,
           location= c("median"), trim.alpha = 0.25)

In [ ]:
# check normality and hormogeneity of the depended variable
p<- ggplot()
p<- p + ylab("frequency")
p<- p + xlab("wildlife_density")
p<- p + theme(text=element_text(size=15))
p<- p + theme(panel.background=element_blank())
p<- p + theme(panel.border=element_rect(fill = NA,colour="black",size=1))
p<- p + theme(strip.background = element_rect(fill = "white", colour ="white",size= 1))
p<- p + theme(text = element_text(size = 15))
p<- p + theme(legend.position = "none")
p<- p + geom_freqpoly(data = transects_all_density, aes(wildlife_density), bins= 20)
plot(p)

In [ ]:
# percentege of zeros in the response variable
sum(transects_all_density$wildlife_density == 0)*100/nrow(transects_all_density)

In [ ]:
# relationships among the response and explanatory variables 
xyplot(wildlife_density ~ dist_bound | regime, 
       group = regime, data = transects_all_density,
       type = c("p", "smooth"))#,
#        scales = "free")

In [ ]:
# Violin plot using panel = panel.violin
bwplot(wildlife_density ~ regime,  data = transects_all_density,
       panel = panel.violin,
       xlab = "Regimes", ylab = "Wildlife Density",col="grey")

In [ ]:
#####  FIT MODELS

In [ ]:
##### Model 1 with combined taset for mbalageti and Kuka transect
# Null model
tweedie_m1out<-tweedie.profile(wildlife_density~1,do.plot =TRUE, data=transects_all_density,p.vec=seq(1.4, 1.9, by=0.1))
profile1.est <- tweedie_m1out$p.max

model1<-glm(wildlife_density~1,family=tweedie(var.power=profile1.est, link.power=0),data=transects_all_density)
summary(model1)

In [ ]:
##### Validation of model 1

In [ ]:
# plot effects for distance variable
par(mfrow=c(2,2))
dres<-resid(model1) #default residuals
pres<-resid(model1, type="pearson")
qres1<-qresid(model1) # quantile residuals, replipcation 1
qres2<-qresid(model1) # quantile residuals, replication 2
qqnorm(dres, main="Deviance residuals",las=1);qqline(dres)
qqnorm(pres,main="Pearson residulas",las=1);qqline(pres)
qqnorm(qres1, main="Quantile residuals (set 1)",las=1);qqline(qres1) # Ideal for model with tweedie distributions
qqnorm(qres2, main="Quantile residuals (set 2)",las=1);qqline(qres2) # Ideal for model with tweedie distributions

par(mfrow=c(3,2))
plot(qres1~fitted(model1),las=1, xlab="Fitted values",ylab="Quantile residuals")
abline(h=0,lty=2)
plot(cooks.distance(model1),type="h",las=1, ylab="Cook's distance",xlab="Observations")
# plot(qresid(model1)~transects_all_density$dist_bound, las=1,
#             xlab="Distance to the boundary",ylab="Quantile residuals")
# abline(h=0,lty=2)
# plot(qresid(model2)~factor(transects_all_density$regime),las=1,
#             xlab="Regime",ylab="Quantile residuals")
abline(h=0,lty=2)
AICtweedie(model1, dispersion=NULL, k= 2,verbose=TRUE) 

In [ ]:
#### Model 2 with combined taset for mbalageti and Kuka Transect

In [ ]:
tweedie_m2out<-tweedie.profile(wildlife_density~dist_bound,do.plot =TRUE, data=transects_all_density,p.vec=seq(1.4, 1.9, by=0.1))
profile2.est <- tweedie_m2out$p.max

model2<-glm(wildlife_density~dist_bound,family=tweedie(var.power=profile2.est, link.power=0),data=transects_all_density)
summary(model2)

In [ ]:
##### Validation of model 2

In [ ]:
# plot effects for distance variable
par(mfrow=c(2,2))
dres<-resid(model2) #default residuals
pres<-resid(model2, type="pearson")
qres1<-qresid(model2) # quantile residuals, replipcation 1
qres2<-qresid(model2) # quantile residuals, replication 2
qqnorm(dres, main="Deviance residuals",las=1);qqline(dres)
qqnorm(pres,main="Pearson residulas",las=1);qqline(pres)
qqnorm(qres1, main="Quantile residuals (set 1)",las=1);qqline(qres1) # Ideal for glm model with tweedie distributions
qqnorm(qres2, main="Quantile residuals (set 2)",las=1);qqline(qres2) # Ideal for glm model with tweedie distributions

par(mfrow=c(3,2))
plot(qres1~fitted(model2),las=1, xlab="Fitted values",ylab="Quantile residuals")
abline(h=0,lty=2)
plot(cooks.distance(model2),type="h",las=1, ylab="Cook's distance",xlab="Observations",ylim=c(0.0,0.15))
plot(qresid(model2)~transects_all_density$dist_bound, las=1,
            xlab="Distance to the boundary",ylab="Quantile residuals")
abline(h=0,lty=2)
# plot(qresid(model2)~factor(transects_all_density$regime),las=1,
#             xlab="Regime",ylab="Quantile residuals")
abline(h=0,lty=2)
AICtweedie(model2, dispersion=NULL, k= 2,verbose=TRUE) 

In [ ]:
# plot effects for distance variable
plot(effect("dist_bound", model2), main="Distance to the boundary")

In [ ]:
# Check for diapersion
disp<-model2$deviance/ model2$df.residual
print(disp)

In [ ]:
#### Model 3 with combined taset for mbalageti and Kuka Transect

In [ ]:
tweedie_m3out<-tweedie.profile(wildlife_density~dist_bound + regime,do.plot =TRUE,
                               data=transects_all_density,p.vec=seq(1.4, 1.9, by=0.1))
profile3.est <- tweedie_m3out$p.max

model3<-glm(wildlife_density~dist_bound + regime ,family=tweedie(var.power=profile3.est, link.power=0),
            data=transects_all_density)
summary(model3)

In [ ]:
##### Validation of model 3

In [ ]:
# plot effects for distance variable
par(mfrow=c(2,2))
dres<-resid(model3) #default residuals
pres<-resid(model3, type="pearson")
qres1<-qresid(model3) # quantile residuals, replipcation 1
qres2<-qresid(model3) # quantile residuals, replication 2
qqnorm(dres, main="Deviance residuals",las=1);qqline(dres)
qqnorm(pres,main="Pearson residulas",las=1);qqline(pres)
qqnorm(qres1, main="Quantile residuals (set 1)",las=1);qqline(qres1) # Ideal for model with tweedie distributions
qqnorm(qres2, main="Quantile residuals (set 2)",las=1);qqline(qres2) # Ideal for model with tweedie distributions

par(mfrow=c(3,2))
plot(qres1~fitted(model3),las=1, xlab="Fitted values",ylab="Quantile residuals")
abline(h=0,lty=2)
plot(cooks.distance(model3),type="h",las=1, ylab="Cook's distance",xlab="Observations")
plot(qresid(model3)~transects_all_density$dist_bound, las=1,
            xlab="Distance to the boundary",ylab="Quantile residuals")
abline(h=0,lty=2)
plot(qresid(model3)~factor(transects_all_density$regime),las=1,
            xlab="Regime",ylab="Quantile residuals")
abline(h=0,lty=2)
AICtweedie(model3, dispersion=NULL, k= 2,verbose=TRUE)    

In [ ]:
# plot effects for distance variable
plot(effect("dist_bound", model3), main="Distance to the boundary")

In [ ]:
# Check for diapersion
disp<-model3$deviance/ model3$df.residual
print(disp)

In [ ]:
#### Model 4 with combined taset for mbalageti and Kuka Transect

In [ ]:
tweedie_m4out<-tweedie.profile(wildlife_density~dist_bound + regime +(dist_bound*regime),do.plot =TRUE,
                               data=transects_all_density,p.vec=seq(1.4, 1.9, by=0.1))
profile4.est <- tweedie_m4out$p.max

model4<-glm(wildlife_density~dist_bound + regime +(dist_bound*regime),family=tweedie(var.power=profile4.est, link.power=0),
            data=transects_all_density)
summary(model4)

In [ ]:
##### Validation of model 4

In [ ]:
# plot effects for distance variable
par(mfrow=c(2,2))
dres<-resid(model4) #default residuals
pres<-resid(model4, type="pearson")
qres1<-qresid(model4) # quantile residuals, replipcation 1
qres2<-qresid(model4) # quantile residuals, replication 2
qqnorm(dres, main="Deviance residuals",las=1);qqline(dres)
qqnorm(pres,main="Pearson residulas",las=1);qqline(pres)
qqnorm(qres1, main="Quantile residuals (set 1)",las=1);qqline(qres1) # Ideal for model with tweedie distributions
qqnorm(qres2, main="Quantile residuals (set 2)",las=1);qqline(qres2) # Ideal for model with tweedie distributions

par(mfrow=c(3,2))
plot(qres1~fitted(model4),las=1, xlab="Fitted values",ylab="Quantile residuals")
abline(h=0,lty=2)
plot(cooks.distance(model4),type="h",las=1, ylab="Cook's distance",xlab="Observations",ylim=c(0.00,0.07))
plot(qresid(model4)~transects_all_density$dist_bound, las=1,
            xlab="Distance to the boundary",ylab="Quantile residuals")
abline(h=0,lty=2)
plot(qresid(model4)~factor(transects_all_density$regime),las=1,
            xlab="Regime",ylab="Quantile residuals")
abline(h=0,lty=2)
AICtweedie(model4, dispersion=NULL, k= 2,verbose=TRUE)  

In [ ]:
# plot effects for distance variable
plot(effect("dist_bound*regime", model4), main="Distance to the boundary * regime effect plot")

In [ ]:
# Check for diapersion
disp<-model4$deviance/ model4$df.residual
print(disp)